In [ ]:
! pip install backtesting yfinance pandas --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 3.4 MB/s eta 0:00:00


In [ ]:
# --- imports ---
from backtesting import Backtest, Strategy, set_bokeh_output
from backtesting.lib import crossover
import yfinance as yf
import pandas as pd

set_bokeh_output(notebook=False)          # PNG plots, no JS warning

# ---------- helper ----------
def fetch_ohlcv(sym, start, end):
    df = yf.Ticker(sym).history(start=start, end=end, auto_adjust=False)
    return df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()

# ---------- indicator ----------
def sma_np(array, n):
    """Simple moving average that works on backtesting.py’s _Array."""
    return pd.Series(array).rolling(n).mean().to_numpy()

# ---------- strategy ----------
class SmaCross(Strategy):
    fast = 50
    slow = 200

    def init(self):
        self.sma_fast = self.I(sma_np, self.data.Close, self.fast)
        self.sma_slow = self.I(sma_np, self.data.Close, self.slow)

    def next(self):
        if crossover(self.sma_fast, self.sma_slow):
            self.buy()
        elif crossover(self.sma_slow, self.sma_fast):
            self.position.close()

# ---------- run ----------
if __name__ == '__main__':
    data = fetch_ohlcv("AAPL", "2020-01-01", "2025-06-17")

    bt = Backtest(data,
                  SmaCross,
                  cash=10_000,
                  commission=0.002,
                  exclusive_orders=True)

    stats = bt.run()
    print(stats)
    bt.plot()


Backtest.run:   0%|          | 0/1171 [00:00<?, ?bar/s]

Start                     2020-01-02 00:00...
End                       2025-06-16 00:00...
Duration                   1991 days 23:00:00
Exposure Time [%]                    32.96864
Equity Final [$]                   8900.91307
Equity Peak [$]                    12317.5025
Commissions [$]                     115.46672
Return [%]                          -10.99087
Buy & Hold Return [%]                64.37743
Return (Ann.) [%]                    -2.11736
Volatility (Ann.) [%]                12.93273
CAGR [%]                             -1.46216
Sharpe Ratio                         -0.16372
Sortino Ratio                        -0.20639
Calmar Ratio                         -0.07155
Alpha [%]                           -21.60879
Beta                                  0.16493
Max. Drawdown [%]                   -29.59463
Avg. Drawdown [%]                    -4.70578
Max. Drawdown Duration      361 days 00:00:00
Avg. Drawdown Duration       54 days 00:00:00
# Trades                          

/usr/local/lib/python3.11/dist-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))
